In [15]:
from glob import iglob
from pprint import pprint
from functools import lru_cache

from pymongo import MongoClient
from dltools import LmaReader

In [16]:
aqlist = range(28, 36 + 1)


@lru_cache()
def taginfo(aq: int) -> dict:
    files = sorted(iglob(f"/helium/exp/SACLA/2017B8065/lma_files/aq{aq:03d}__*.lma"))

    with LmaReader(files[0]) as f:
        tags = (d['tag'] for d in f)
        fr = min(tags)

    with LmaReader(files[-1]) as f:
        tags = (d['tag'] for d in f)
        to = max(tags)
    
    return {"aq": aq, "fr": fr, "to": to}


pprint([taginfo(aq) for aq in aqlist])

[{'aq': 28, 'fr': 156993292, 'to': 157299895},
 {'aq': 29, 'fr': 157304892, 'to': 157417387},
 {'aq': 30, 'fr': 157531677, 'to': 157795488},
 {'aq': 31, 'fr': 157807353, 'to': 157818044},
 {'aq': 32, 'fr': 157838034, 'to': 157949434},
 {'aq': 33, 'fr': 157962252, 'to': 158177368},
 {'aq': 34, 'fr': 158180227, 'to': 158623041},
 {'aq': 35, 'fr': 158648231, 'to': 158823296},
 {'aq': 36, 'fr': 158925249, 'to': 158977947}]


In [18]:
print("Update data...")
with MongoClient("mongodb") as client:
    collect = client["sacla_2017b8065"]["resorted"]

    for aq in aqlist:
        d = taginfo(aq)
        collect.update_many(
            {"tag": {"$gte": d["fr"], "$lte": d["to"]}},
            {"$set": {"aq": d["aq"]}},
        )
print("Done!")

Update data...
Done!
